# Bahdanau 注意力（Bahdanau Attention）

Bahdanau 注意力（Bahdanau Attention）是一种经典的注意力机制，由 Bahdanau 等人在 2014 年提出，用于改进传统的序列到序列（Seq2Seq）模型。Bahdanau 注意力通过引入加性注意力（Additive Attention）机制，使得模型能够在生成输出序列时动态地关注输入序列的不同部分，从而提高生成结果的质量。

## 1. 背景

在传统的 Seq2Seq 模型中，编码器将整个输入序列压缩成一个固定大小的上下文向量（Context Vector），然后解码器根据这个向量生成输出序列。然而，这种压缩可能导致信息的丢失，尤其是在处理长序列时。

## 2. Bahdanau 注意力的核心思想

Bahdanau 注意力的核心思想是通过计算解码器当前隐藏状态与编码器所有隐藏状态之间的相似度，来动态地关注输入序列的不同部分。具体来说，Bahdanau 注意力使用加性注意力机制来计算注意力分数，并根据这些分数对输入序列进行加权求和，得到加权上下文向量。

## 3. 工作原理

### 3.1 编码器

编码器将输入序列转换为一系列隐藏状态（Hidden States），每个隐藏状态对应输入序列中的一个元素。这些隐藏状态既作为键，也作为值。

### 3.2 解码器

在解码器的每一步生成过程中，Bahdanau 注意力执行以下步骤：

1. **计算注意力分数**：解码器当前的隐藏状态（查询）与编码器的每个隐藏状态（键）进行比较，计算它们之间的相似度。Bahdanau 注意力使用加性注意力机制，通过一个前馈神经网络（Feedforward Neural Network）来计算注意力分数。
   - **公式**：
     \[
     \text{score}(Q, K) = \text{FFN}(\text{concat}(Q, K))
     \]
     其中，\( Q \) 是解码器的当前隐藏状态（查询），\( K \) 是编码器的隐藏状态（键），FFN 是一个前馈神经网络。

2. **归一化**：对注意力分数进行归一化处理（如使用 Softmax 函数），得到注意力权重。
   - **公式**：
     \[
     \alpha_i = \text{softmax}(\text{score}(Q, K_i))
     \]
     其中，\( \alpha_i \) 是第 \( i \) 个输入元素的注意力权重。

3. **加权求和**：根据注意力权重对编码器的隐藏状态（值）进行加权求和，得到加权上下文向量。
   - **公式**：
     \[
     \text{context} = \sum_i \alpha_i \cdot V_i
     \]
     其中，\( V_i \) 是第 \( i \) 个输入元素的值向量。

4. **生成输出**：将加权上下文向量与解码器的当前隐藏状态结合，生成当前的输出元素。

![S2S](https://zh-v2.d2l.ai/_images/seq2seq-attention-details.svg "S2S")

## 4. 优点与局限性

### 4.1 优点

- **动态关注**：Bahdanau 注意力允许模型在每一步生成过程中动态地关注输入序列的不同部分，从而更好地捕捉输入序列中的重要信息。
- **处理长序列**：相比于传统的 Seq2Seq 模型，Bahdanau 注意力在处理长序列时表现更好，因为它不需要将整个输入序列压缩成一个固定大小的向量。

### 4.2 局限性

- **计算复杂度**：Bahdanau 注意力的计算复杂度较高，尤其是在处理长序列时。
- **可解释性**：虽然 Bahdanau 注意力提高了模型的性能，但它也使得模型的可解释性降低，因为注意力权重是动态计算的，难以直观理解。

## 5. 应用场景

- **机器翻译**：在生成翻译后的句子时，Bahdanau 注意力可以帮助模型关注源语言句子中的重要部分。
- **文本摘要**：在生成摘要时，Bahdanau 注意力可以帮助模型关注原文中的关键信息。
- **问答系统**：在生成回答时，Bahdanau 注意力可以帮助模型关注问题中的关键部分和相关上下文。

## 6. 总结

Bahdanau 注意力是一种经典的注意力机制，通过引入加性注意力机制，使得模型能够在生成输出序列时动态地关注输入序列的不同部分，从而提高生成结果的质量。尽管计算复杂度较高，但 Bahdanau 注意力在许多 NLP 任务中表现出色，成为现代深度学习模型的核心组件之一。

## 简单代码实现

首先，初始化解码器的状态，需要下面的输入：

1. 编码器在所有时间步的最终层隐状态，将作为注意力的键和值；

2. 上一时间步的编码器全层隐状态，将作为初始化解码器的隐状态；

3. 编码器有效长度（排除在注意力池中填充词元）。

In [ ]:
class Seq2SeqAttentionDecoder(AttentionDecoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqAttentionDecoder, self).__init__(**kwargs)
        self.attention = d2l.AdditiveAttention(
            num_hiddens, num_hiddens, num_hiddens, dropout)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(
            embed_size + num_hiddens, num_hiddens, num_layers,
            dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        # outputs的形状为(batch_size，num_steps，num_hiddens).
        # hidden_state的形状为(num_layers，batch_size，num_hiddens)
        outputs, hidden_state = enc_outputs
        return (outputs.permute(1, 0, 2), hidden_state, enc_valid_lens)

    def forward(self, X, state):
        # enc_outputs的形状为(batch_size,num_steps,num_hiddens).
        # hidden_state的形状为(num_layers,batch_size,
        # num_hiddens)
        enc_outputs, hidden_state, enc_valid_lens = state
        # 输出X的形状为(num_steps,batch_size,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        outputs, self._attention_weights = [], []
        for x in X:
            # query的形状为(batch_size,1,num_hiddens)
            query = torch.unsqueeze(hidden_state[-1], dim=1)
            # context的形状为(batch_size,1,num_hiddens)
            context = self.attention(
                query, enc_outputs, enc_outputs, enc_valid_lens)
            # 在特征维度上连结
            x = torch.cat((context, torch.unsqueeze(x, dim=1)), dim=-1)
            # 将x变形为(1,batch_size,embed_size+num_hiddens)
            out, hidden_state = self.rnn(x.permute(1, 0, 2), hidden_state)
            outputs.append(out)
            self._attention_weights.append(self.attention.attention_weights)
        # 全连接层变换后，outputs的形状为
        # (num_steps,batch_size,vocab_size)
        outputs = self.dense(torch.cat(outputs, dim=0))
        return outputs.permute(1, 0, 2), [enc_outputs, hidden_state,
                                          enc_valid_lens]

    @property
    def attention_weights(self):
        return self._attention_weights